<a href="https://colab.research.google.com/github/SamiraHabib/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera Capstone #

## Modelo de negócio: ##

### Verificar o melhor local onde se encontram o maior número de escolas e o menor número de papelarias. A finalidade seria montar uma papelaria num lugar propício que possa atingir um maior público que utiliza esses serviços. ###

# Importando as bibliotecas #

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 81kB 5.0MB/s 
  Created wheel for folium: filename=folium-0.5.0-cp37-none-any.whl size=76240 sha256=63271da22701ffe195a78f5390135233317617aae82c2725ffd57ea879ba97d1
  Stored in directory: /root/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.5.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Folium installed
Libraries imported.


# Definição das credenciais e version do Foursquare #

In [2]:
CLIENT_ID = '1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA' # your Foursquare ID
CLIENT_SECRET = 'ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA
CLIENT_SECRET:ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D


# Endereço do local escolhido #

In [124]:
#Exemplo - alterar
address = 'Rio de Janeiro, Maracanã'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-22.912161949999998 -43.23117510590559


# Localização #

In [125]:
#Localidade exemplo, mudar
latitude = -22.912161949999998 
longitude = -43.23117510590559

# Define URL #

In [126]:
search_query = 'colégio'
radius = 2000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

colégio .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA&client_secret=ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D&ll=-22.912161949999998,-43.23117510590559&v=20180604&query=colégio&radius=2000&limit=50'

# Envia uma solicitação GET e examina os resultados #

In [127]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6082ed5278a23a2b539cdf69'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d13b941735',
      'name': 'School',
      'pluralName': 'Schools',
      'primary': True,
      'shortName': 'School'}],
    'hasPerk': False,
    'id': '4dc1528422713750ba79c2ad',
    'location': {'address': 'R. S. Francisco Xavier, 267',
     'cc': 'BR',
     'city': 'Rio de Janeiro',
     'country': 'Brasil',
     'distance': 660,
     'formattedAddress': ['R. S. Francisco Xavier, 267',
      'Rio de Janeiro, RJ',
      '20550-010',
      'Brasil'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.916797220457074,
       'lng': -43.227150214751354}],
     'lat': -22.916797220457074,
     'lng': -43.227150214751354,
     'postalCode': '20550-010',
     'state': 'RJ'},
    'name': 'Colégio Militar do Rio de Janeiro (CMRJ)',
    'referra

## Pegando a parte relevante do JSON e transformando em um dataframe (pandas) ##

In [128]:
#transformando em dataframe
venues = results['response']['venues']
df_school = json_normalize(venues)
df_school.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4dc1528422713750ba79c2ad,Colégio Militar do Rio de Janeiro (CMRJ),"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1619193170,False,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",660,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",NaN
1,4dbfe92c4b2221ec2d640bac,Colégio Batista,[],v-1619193170,False,"Rua Visconde de Itamarati, 75",-22.916780,-43.231521,"[{'label': 'display', 'lat': -22.9167803740600...",515,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Visconde de Itamarati, 75 (Maracanã), Rio...",Maracanã
2,512b53c1e4b0fd3b1159b204,Colégio Ressurreição,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1619193170,False,"Rua Oto de Alencar, 23",-22.915196,-43.225821,"[{'label': 'display', 'lat': -22.9151963527452...",644,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Oto de Alencar, 23, Rio de Janeiro, RJ, B...",NaN
3,5071a03ce4b0e1b65ea15489,Colégio Pedro II,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1619193170,False,"R. S. Francisco Xavier, 204/208",-22.916347,-43.225898,"[{'label': 'display', 'lat': -22.9163471631107...",714,20550-012,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 204/208 (Campus Tijuc...",Campus Tijuca II
4,4ea5556577c8d0ce5f233bbb,Colégio Colégio Nossa Senhora de Lourdes (NSL),"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619193170,False,"R. Oito de Dezembro, 328",-22.909630,-43.240261,"[{'label': 'display', 'lat': -22.9096297441251...",973,20550-201,BR,Rio de Janeiro,RJ,Brasil,"[R. Oito de Dezembro, 328, Rio de Janeiro, RJ,...",NaN


# Define a informação de interesse e filtra o dataframe #

In [129]:
# Mantém somente as colunas que incluem venue name e qualquer coisa que é associada com a localização
filtered_columns = ['name', 'categories'] + [col for col in df_school.columns if col.startswith('location.')] + ['id']
df_school_filtered = df_school.loc[:, filtered_columns]

# função que extrai a categoria de venue 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtra a categoria por cada linha do data frame
df_school_filtered['categories'] = df_school_filtered.apply(get_category_type, axis=1)

# limpa os nomes das colunas para manter somente o último termo
df_school_filtered.columns = [column.split('.')[-1] for column in df_school_filtered.columns]

df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Colégio Militar do Rio de Janeiro (CMRJ),School,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",660,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",NaN,4dc1528422713750ba79c2ad
1,Colégio Batista,None,"Rua Visconde de Itamarati, 75",-22.916780,-43.231521,"[{'label': 'display', 'lat': -22.9167803740600...",515,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Visconde de Itamarati, 75 (Maracanã), Rio...",Maracanã,4dbfe92c4b2221ec2d640bac
2,Colégio Ressurreição,School,"Rua Oto de Alencar, 23",-22.915196,-43.225821,"[{'label': 'display', 'lat': -22.9151963527452...",644,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Oto de Alencar, 23, Rio de Janeiro, RJ, B...",NaN,512b53c1e4b0fd3b1159b204
3,Colégio Pedro II,School,"R. S. Francisco Xavier, 204/208",-22.916347,-43.225898,"[{'label': 'display', 'lat': -22.9163471631107...",714,20550-012,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 204/208 (Campus Tijuc...",Campus Tijuca II,5071a03ce4b0e1b65ea15489
4,Colégio Colégio Nossa Senhora de Lourdes (NSL),Private School,"R. Oito de Dezembro, 328",-22.909630,-43.240261,"[{'label': 'display', 'lat': -22.9096297441251...",973,20550-201,BR,Rio de Janeiro,RJ,Brasil,"[R. Oito de Dezembro, 328, Rio de Janeiro, RJ,...",NaN,4ea5556577c8d0ce5f233bbb


# Avaliação dos dados: #

In [130]:
df_school_filtered.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
cc                   object
city                 object
state                object
country              object
formattedAddress     object
crossStreet          object
id                   object
dtype: object

In [131]:
df_school_filtered.describe()

,lat,lng,distance
count,50.000000,50.000000,50.000000
mean,-22.916834,-43.230304,1187.960000
std,0.006701,0.008836,469.424527
min,-22.931675,-43.246514,221.000000
25%,-22.921262,-43.238463,807.500000
50%,-22.916354,-43.230589,1236.500000
75%,-22.914015,-43.222731,1550.250000
max,-22.899421,-43.214707,2213.000000


In [132]:
df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Colégio Militar do Rio de Janeiro (CMRJ),School,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",660,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",NaN,4dc1528422713750ba79c2ad
1,Colégio Batista,None,"Rua Visconde de Itamarati, 75",-22.916780,-43.231521,"[{'label': 'display', 'lat': -22.9167803740600...",515,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Visconde de Itamarati, 75 (Maracanã), Rio...",Maracanã,4dbfe92c4b2221ec2d640bac
2,Colégio Ressurreição,School,"Rua Oto de Alencar, 23",-22.915196,-43.225821,"[{'label': 'display', 'lat': -22.9151963527452...",644,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Rua Oto de Alencar, 23, Rio de Janeiro, RJ, B...",NaN,512b53c1e4b0fd3b1159b204
3,Colégio Pedro II,School,"R. S. Francisco Xavier, 204/208",-22.916347,-43.225898,"[{'label': 'display', 'lat': -22.9163471631107...",714,20550-012,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 204/208 (Campus Tijuc...",Campus Tijuca II,5071a03ce4b0e1b65ea15489
4,Colégio Colégio Nossa Senhora de Lourdes (NSL),Private School,"R. Oito de Dezembro, 328",-22.909630,-43.240261,"[{'label': 'display', 'lat': -22.9096297441251...",973,20550-201,BR,Rio de Janeiro,RJ,Brasil,"[R. Oito de Dezembro, 328, Rio de Janeiro, RJ,...",NaN,4ea5556577c8d0ce5f233bbb


# Número de nulos das colunas #

In [133]:
print("Number of NaN values for the column city :", df_school_filtered['city'].isnull().sum())
print("Number of NaN values for the column state :", df_school_filtered['state'].isnull().sum())
print("Number of NaN values for the column crossStreet :", df_school_filtered['crossStreet'].isnull().sum())

Number of NaN values for the column city : 14
Number of NaN values for the column state : 14
Number of NaN values for the column crossStreet : 45


In [134]:
df_school_filtered['state'].replace(np.nan,"RJ", inplace=True)

In [135]:
df_school_filtered['city'].replace(np.nan,"Rio de Janeiro", inplace=True)

In [136]:
print("Number of NaN values for the column state :", df_school_filtered['state'].isnull().sum())
print("Number of NaN values for the column city :", df_school_filtered['city'].isnull().sum())

Number of NaN values for the column state : 0
Number of NaN values for the column city : 0


In [137]:
df_school_filtered['categories'].value_counts()

School                             13
High School                        10
Private School                      7
College Classroom                   3
Student Center                      3
College Academic Building           2
College Administrative Building     2
Community College                   2
Daycare                             1
Religious School                    1
Church                              1
University                          1
Elementary School                   1
General College & University        1
Name: categories, dtype: int64

In [138]:
df_school_filtered.isnull().sum()

name                 0
categories           2
address             14
lat                  0
lng                  0
labeledLatLngs       0
distance             0
postalCode          28
cc                   0
city                 0
state                0
country              0
formattedAddress     0
crossStreet         45
id                   0
dtype: int64

# Dropando uma coluna #

In [139]:
df_school_filtered.drop('crossStreet', axis=1, inplace=True)
df_school_filtered.drop('postalCode', axis=1, inplace=True)
df_school_filtered.drop('cc', axis=1, inplace=True)
df_school_filtered.drop('city', axis=1, inplace=True)
df_school_filtered.drop('state', axis=1, inplace=True)
df_school_filtered.drop('country', axis=1, inplace=True)

In [140]:
df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,formattedAddress,id
0,Colégio Militar do Rio de Janeiro (CMRJ),School,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",660,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",4dc1528422713750ba79c2ad
1,Colégio Batista,None,"Rua Visconde de Itamarati, 75",-22.916780,-43.231521,"[{'label': 'display', 'lat': -22.9167803740600...",515,"[Rua Visconde de Itamarati, 75 (Maracanã), Rio...",4dbfe92c4b2221ec2d640bac
2,Colégio Ressurreição,School,"Rua Oto de Alencar, 23",-22.915196,-43.225821,"[{'label': 'display', 'lat': -22.9151963527452...",644,"[Rua Oto de Alencar, 23, Rio de Janeiro, RJ, B...",512b53c1e4b0fd3b1159b204
3,Colégio Pedro II,School,"R. S. Francisco Xavier, 204/208",-22.916347,-43.225898,"[{'label': 'display', 'lat': -22.9163471631107...",714,"[R. S. Francisco Xavier, 204/208 (Campus Tijuc...",5071a03ce4b0e1b65ea15489
4,Colégio Colégio Nossa Senhora de Lourdes (NSL),Private School,"R. Oito de Dezembro, 328",-22.909630,-43.240261,"[{'label': 'display', 'lat': -22.9096297441251...",973,"[R. Oito de Dezembro, 328, Rio de Janeiro, RJ,...",4ea5556577c8d0ce5f233bbb


In [141]:
df_school_filtered['name'].count()

50

In [142]:
print('There are {} uniques categories.'.format(len(df_school_filtered['categories'].unique())))

There are 15 uniques categories.


# Nome e categorias dos dados #

In [143]:
df_school_filtered.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'formattedAddress', 'id'],
      dtype='object')

In [144]:
df_school_filtered.categories

0                              School
1                                None
2                              School
3                              School
4                      Private School
5                              School
6                         High School
7                      Private School
8                      Private School
9                      Private School
10                        High School
11                             School
12                        High School
13                  College Classroom
14                             School
15                     Private School
16       General College & University
17                   Religious School
18                        High School
19                        High School
20                     Student Center
21                  Community College
22                        High School
23                        High School
24                  College Classroom
25                             School
26          

# Identificar as ids de escolas diferentes #

In [145]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# display map
venues_map

# Visualizar escolas no mapa ao redor da localização anteriormente citada #

In [146]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# Escolas em azul ao entorno
for lat, lng, label in zip(df_school_filtered.lat, df_school_filtered.lng, df_school_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
       # popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Novo dataframe para fazer um merge #

In [147]:
CLIENT_ID = '1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA' # your Foursquare ID
CLIENT_SECRET = 'ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA
CLIENT_SECRET:ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D


In [148]:
#Exemplo - alterar
address = 'Rio de Janeiro, Maracanã'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-22.912161949999998 -43.23117510590559


In [149]:
#Localidade exemplo, mudar
latitude = -22.912161949999998
longitude = -43.23117510590559

In [150]:
search_query = 'papelaria'
radius = 2000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

papelaria .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA&client_secret=ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D&ll=-22.912161949999998,-43.23117510590559&v=20180604&query=papelaria&radius=2000&limit=50'

In [151]:
import requests
negocios = requests.get(url).json()
negocios

{'meta': {'code': 200, 'requestId': '6082edb230b264432c582e9b'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/papergoods_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d121951735',
      'name': 'Paper / Office Supplies Store',
      'pluralName': 'Paper / Office Supplies Stores',
      'primary': True,
      'shortName': 'Office Supplies'}],
    'hasPerk': False,
    'id': '4e946c658b81ef41a94b7377',
    'location': {'address': 'Rua São Francisco Xavier',
     'cc': 'BR',
     'city': 'Rio de Janeiro',
     'country': 'Brasil',
     'distance': 427,
     'formattedAddress': ['Rua São Francisco Xavier',
      'Rio de Janeiro, RJ',
      'Brasil'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.91596947304269,
       'lng': -43.23062765107665}],
     'lat': -22.91596947304269,
     'lng': -43.23062765107665,
     'state': 'RJ'},
    'name': 'Papelaria Porto Seguro',
    'referralId': 'v-1619193266'},


In [157]:
#transformando em dataframe
venues = negocios['response']['venues']
df_paper = json_normalize(venues)
df_paper.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,venuePage.id
0,4e946c658b81ef41a94b7377,Papelaria Porto Seguro,"[{'id': '4bf58dd8d48988d121951735', 'name': 'P...",v-1619193266,False,Rua São Francisco Xavier,-22.915969,-43.230628,"[{'label': 'display', 'lat': -22.9159694730426...",427,BR,Rio de Janeiro,RJ,Brasil,"[Rua São Francisco Xavier, Rio de Janeiro, RJ,...",NaN,NaN,NaN
1,4f6258a3e4b0832f00d7d3cd,Papelaria Yunes,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",v-1619193266,False,NaN,-22.912098,-43.236651,"[{'label': 'display', 'lat': -22.9120980467721...",561,BR,NaN,NaN,Brasil,[Brasil],NaN,NaN,NaN
2,549da21b498e520a4e30b6b6,Papelaria Papel Moderno,"[{'id': '52f2ab2ebcbc57f1066b8b21', 'name': 'S...",v-1619193266,False,"R. Sto. Afonso, 101",-22.922829,-43.231915,"[{'label': 'display', 'lat': -22.9228290454082...",1189,BR,Rio de Janeiro,RJ,Brasil,"[R. Sto. Afonso, 101, Rio de Janeiro, RJ, Brasil]",NaN,NaN,NaN
3,53c547c1498ef1a0284198aa,papelaria raposo,"[{'id': '52f2ab2ebcbc57f1066b8b1e', 'name': 'L...",v-1619193266,False,NaN,-22.919445,-43.234497,"[{'label': 'display', 'lat': -22.9194454163645...",879,BR,NaN,NaN,Brasil,[Brasil],NaN,NaN,NaN
4,4ea69f80cc219396a47722c6,Papelaria Risco e Rabisco,"[{'id': '4bf58dd8d48988d121951735', 'name': 'P...",v-1619193266,False,"R. Mariz e Barros, 653 A",-22.914617,-43.219935,"[{'label': 'display', 'lat': -22.9146166683652...",1184,BR,Rio de Janeiro,RJ,Brasil,"[R. Mariz e Barros, 653 A, Rio de Janeiro, RJ,...",NaN,NaN,NaN


In [158]:
# Mantém somente as colunas que incluem venue name e qualquer coisa que é associada com a localização
filtered_columns = ['name', 'categories'] + [col for col in df_paper.columns if col.startswith('location.')] + ['id']
df_paper_filtro = df_paper.loc[:, filtered_columns]

# função que extrai a categoria de venue 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtra a categoria por cada linha do data frame
df_paper_filtro['categories'] = df_paper_filtro.apply(get_category_type, axis=1)

# limpa os nomes das colunas para manter somente o último termo
df_paper_filtro.columns = [column.split('.')[-1] for column in df_paper_filtro.columns]

df_paper_filtro.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Papelaria Porto Seguro,Paper / Office Supplies Store,Rua São Francisco Xavier,-22.915969,-43.230628,"[{'label': 'display', 'lat': -22.9159694730426...",427,BR,Rio de Janeiro,RJ,Brasil,"[Rua São Francisco Xavier, Rio de Janeiro, RJ,...",NaN,NaN,4e946c658b81ef41a94b7377
1,Papelaria Yunes,Bookstore,NaN,-22.912098,-43.236651,"[{'label': 'display', 'lat': -22.9120980467721...",561,BR,NaN,NaN,Brasil,[Brasil],NaN,NaN,4f6258a3e4b0832f00d7d3cd
2,Papelaria Papel Moderno,Stationery Store,"R. Sto. Afonso, 101",-22.922829,-43.231915,"[{'label': 'display', 'lat': -22.9228290454082...",1189,BR,Rio de Janeiro,RJ,Brasil,"[R. Sto. Afonso, 101, Rio de Janeiro, RJ, Brasil]",NaN,NaN,549da21b498e520a4e30b6b6
3,papelaria raposo,Locksmith,NaN,-22.919445,-43.234497,"[{'label': 'display', 'lat': -22.9194454163645...",879,BR,NaN,NaN,Brasil,[Brasil],NaN,NaN,53c547c1498ef1a0284198aa
4,Papelaria Risco e Rabisco,Paper / Office Supplies Store,"R. Mariz e Barros, 653 A",-22.914617,-43.219935,"[{'label': 'display', 'lat': -22.9146166683652...",1184,BR,Rio de Janeiro,RJ,Brasil,"[R. Mariz e Barros, 653 A, Rio de Janeiro, RJ,...",NaN,NaN,4ea69f80cc219396a47722c6


In [159]:
df_paper_filtro.isnull().sum()

name                 0
categories           0
address             13
lat                  0
lng                  0
labeledLatLngs       0
distance             0
cc                   0
city                15
state               15
country              0
formattedAddress     0
crossStreet         30
postalCode          31
id                   0
dtype: int64

# Dropando colunas: #

In [160]:
df_paper_filtro.drop('crossStreet', axis=1, inplace=True)
df_paper_filtro.drop('cc', axis=1, inplace=True)
df_paper_filtro.drop('city', axis=1, inplace=True)
df_paper_filtro.drop('state', axis=1, inplace=True)
df_paper_filtro.drop('country', axis=1, inplace=True)
df_paper_filtro.drop('postalCode', axis=1, inplace=True)

In [161]:
df_paper_filtro.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,formattedAddress,id
0,Papelaria Porto Seguro,Paper / Office Supplies Store,Rua São Francisco Xavier,-22.915969,-43.230628,"[{'label': 'display', 'lat': -22.9159694730426...",427,"[Rua São Francisco Xavier, Rio de Janeiro, RJ,...",4e946c658b81ef41a94b7377
1,Papelaria Yunes,Bookstore,NaN,-22.912098,-43.236651,"[{'label': 'display', 'lat': -22.9120980467721...",561,[Brasil],4f6258a3e4b0832f00d7d3cd
2,Papelaria Papel Moderno,Stationery Store,"R. Sto. Afonso, 101",-22.922829,-43.231915,"[{'label': 'display', 'lat': -22.9228290454082...",1189,"[R. Sto. Afonso, 101, Rio de Janeiro, RJ, Brasil]",549da21b498e520a4e30b6b6
3,papelaria raposo,Locksmith,NaN,-22.919445,-43.234497,"[{'label': 'display', 'lat': -22.9194454163645...",879,[Brasil],53c547c1498ef1a0284198aa
4,Papelaria Risco e Rabisco,Paper / Office Supplies Store,"R. Mariz e Barros, 653 A",-22.914617,-43.219935,"[{'label': 'display', 'lat': -22.9146166683652...",1184,"[R. Mariz e Barros, 653 A, Rio de Janeiro, RJ,...",4ea69f80cc219396a47722c6


In [162]:
df_paper_filtro.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'formattedAddress', 'id'],
      dtype='object')

# Novo  dataframe (juntando os dataframes) # 

In [163]:
df_new = df_paper_filtro.append(df_school_filtered)

In [164]:
df_new.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,formattedAddress,id
0,Papelaria Porto Seguro,Paper / Office Supplies Store,Rua São Francisco Xavier,-22.915969,-43.230628,"[{'label': 'display', 'lat': -22.9159694730426...",427,"[Rua São Francisco Xavier, Rio de Janeiro, RJ,...",4e946c658b81ef41a94b7377
1,Papelaria Yunes,Bookstore,NaN,-22.912098,-43.236651,"[{'label': 'display', 'lat': -22.9120980467721...",561,[Brasil],4f6258a3e4b0832f00d7d3cd
2,Papelaria Papel Moderno,Stationery Store,"R. Sto. Afonso, 101",-22.922829,-43.231915,"[{'label': 'display', 'lat': -22.9228290454082...",1189,"[R. Sto. Afonso, 101, Rio de Janeiro, RJ, Brasil]",549da21b498e520a4e30b6b6
3,papelaria raposo,Locksmith,NaN,-22.919445,-43.234497,"[{'label': 'display', 'lat': -22.9194454163645...",879,[Brasil],53c547c1498ef1a0284198aa
4,Papelaria Risco e Rabisco,Paper / Office Supplies Store,"R. Mariz e Barros, 653 A",-22.914617,-43.219935,"[{'label': 'display', 'lat': -22.9146166683652...",1184,"[R. Mariz e Barros, 653 A, Rio de Janeiro, RJ,...",4ea69f80cc219396a47722c6


In [165]:
df_new.drop('address', axis=1, inplace=True)
df_new.drop('labeledLatLngs', axis=1, inplace=True)
df_new.drop('formattedAddress', axis=1, inplace=True)

In [166]:
df_new.head()

,name,categories,lat,lng,distance,id
0,Papelaria Porto Seguro,Paper / Office Supplies Store,-22.915969,-43.230628,427,4e946c658b81ef41a94b7377
1,Papelaria Yunes,Bookstore,-22.912098,-43.236651,561,4f6258a3e4b0832f00d7d3cd
2,Papelaria Papel Moderno,Stationery Store,-22.922829,-43.231915,1189,549da21b498e520a4e30b6b6
3,papelaria raposo,Locksmith,-22.919445,-43.234497,879,53c547c1498ef1a0284198aa
4,Papelaria Risco e Rabisco,Paper / Office Supplies Store,-22.914617,-43.219935,1184,4ea69f80cc219396a47722c6


In [170]:
df_new.tail()

,name,categories,lat,lng,distance,id
45,Colégio e Curso Educação,University,-22.916009,-43.246514,1630,5ce9348aa795bd002c5b39d1
46,colégio batista shepard,Daycare,-22.926686,-43.233873,1640,512e58f3e4b0016f1c127dae
47,Colegio Van Gogh,None,-22.921150,-43.218220,1662,4e64c62db0fb188e8eae5a21
48,Colégio Ícaro.,High School,-22.925388,-43.232258,1476,5204e05a498e64a5b96d47ce
49,Colégio Estadual Affonso Penna,Student Center,-22.924114,-43.241054,1672,52f005b4498e07bc50005368


In [167]:
df_new.shape

(87, 6)

# Visualizar as escolas e papelarias na redondeza do centro #

In [168]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# Escolas em azul ao entorno
for lat, lng, label in zip(df_school_filtered.lat, df_school_filtered.lng, df_school_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# Papelarias em azul ao entorno
for lat, lng, label in zip(df_paper_filtro.lat, df_paper_filtro.lng, df_paper_filtro.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map